In [1]:
import numpy as np
import pandas as pd
from pandas.core.arrays.period import timedelta

import plotly.graph_objs as go

In [2]:
def rename_prices(df):
    if 'PRICES' in df.columns:
        df.rename(columns={"PRICES": "datetime"}, inplace = True)
    else:
        print("There's no column PRICES.")


def dataformatting(df):
    #wide to long
    df = df.melt(id_vars=['datetime'], value_vars=df.columns[1:25]).sort_values(['datetime', 'variable'])
    df.reset_index(inplace=True, drop=True)
    
    #creating master time column, ulgy but works
    time = df['datetime'].copy()
    for d in range(len(df['datetime'])):
        time[d] = df['datetime'][d]+timedelta(hours = d%24) #decided not to go for the +1, so hour 1 is midnight, makes more sense, now it ends in 2009, otherwise the last measurement was 01.01.2010 00:00:00
    df['time'] = time
    
    #hour from string to int
    df['variable'] = df['variable'].map(lambda x:int(x[-2:]))
    
    #renaming, shullfing columns (not important)
    df.rename(columns={"datetime": "date", "variable": "hour", "value":"price"}, inplace = True)
    cols = df.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    df = df[cols]
    
    return df

def drop_n_last_rows(df, n):
    df.drop(df.tail(1).index,inplace=True)
    
def conv(fld):
    if fld.endswith(b']'):
        return float(fld[:-1])
    elif fld.startswith(b'['):
        return float(fld[1:])
    else:
        return float(fld)
    
def load_validation_results(txt_file_name:str='experiment_results.txt', actionBool = True):
    val = pd.read_excel('./data/val_data/validate.xlsx')
    rename_prices(val)
    val = dataformatting(val)
    
    filename = './model/tabular_q/'+txt_file_name
    
    if actionBool:
        # To load the actions (reward/mkt_price)
        results = np.loadtxt(filename, delimiter=',', skiprows=3,max_rows = 1, dtype=float, converters=conv)
    else:
        # To load the rewards
        results = np.loadtxt(filename, delimiter=',', skiprows=1,max_rows = 1, dtype=float, converters=conv)
    
    if len(val)>len(results):
        # Drop the last row from validation data due to lack of action/reward taken when in that state 
        drop_n_last_rows(val, n=1)
    
    val['action'] = results
    
    return val

In [3]:
val=load_validation_results()
val.sample(5)

,time,date,hour,price,action
10760,2011-03-25 08:00:00,2011-03-25,9,66.71,1.05948
9930,2011-02-18 18:00:00,2011-02-18,19,68.41,0.00000
1712,2010-03-13 08:00:00,2010-03-13,9,34.00,0.00000
993,2010-02-11 09:00:00,2010-02-11,10,66.20,0.00000
13624,2011-07-22 16:00:00,2011-07-22,17,47.37,0.00000


In [4]:
def buy_sell_hodl_plot(df, howmanydays=5):
    howmanyhours=howmanydays*24
    max_range=len(df)-howmanyhours
    ri = np.random.randint(0, max_range)

    trace=dict(type='scatter',
              x=df.time[ri:ri+howmanyhours],
              y=df.price[ri:ri+howmanyhours],
              mode='lines+markers',
              marker=dict(color= df.action[ri:ri+howmanyhours], 
                          colorscale='Cividis', size=7, colorbar=dict(thickness=20,tickvals=[df.action[ri:ri+howmanyhours].max(), 0.0, df.action[ri:ri+howmanyhours].min()], ticktext=['Sell', 'HODL','Buy'])))


    axis_style=dict(zeroline=False, showline=True, mirror=True)
    layout=dict(width=900, height=600, 
                title=f'Actions taken vs. energy prices between {df.time[ri].day} {df.time[ri].month_name()} - {df.time[ri+howmanyhours-1].day} {df.time[ri+howmanyhours-1].month_name()} {df.time[ri+howmanyhours-1].year}',
                xaxis=axis_style,
                yaxis=axis_style,
                yaxis_title="Energy market price (Euro/MWh)",
               hovermode='closest', showlegend=False)
    fw=go.FigureWidget(data=[trace], layout=layout)

    fw.show()

In [5]:
buy_sell_hodl_plot(val, 5)